In [1]:
import os
os.chdir("..")

In [2]:
os.getcwd()

'/home/phuc/simone'

In [6]:
import einops as E
import jax.numpy as jnp
import jax
import optax
import elegy as eg

from model import SIMONeModel
from dataloader import build_dataloader

In [7]:
import mediapy as media

In [10]:
train_dl, val_dl = build_dataloader(
    8,
    num_workers=1,
    dataset_class="vor",
    path="data/datasets",
    channel_last=True,
)
train_ds = train_dl.dataset
val_ds = val_dl.dataset

tcmalloc: large alloc 4915200000 bytes == 0x1c45c6000 @  0x7fd6e48ea680 0x7fd6e490b824 0x7fd6c401e4ce 0x7fd6c406fc70 0x7fd6c406fefe 0x7fd6c4108da7 0x503ea9 0x56bf09 0x56a0ba 0x5f6343 0x59cfb7 0x5f68ef 0x57195c 0x56a0ba 0x5f6343 0x56cf2a 0x56a0ba 0x68d5b7 0x600f54 0x5c5530 0x56bddd 0x5004f8 0x56d80c 0x5004f8 0x56d80c 0x5004f8 0x5042c6 0x56bf09 0x5f6166 0x56bddd 0x5f6166


In [11]:
n_vids = 100
nframes = train_ds[0].shape[0]
train_ds[0].shape

(10, 64, 64, 3)

In [12]:
batch = next(iter(train_dl))
batch = batch.numpy()
batch.dtype, batch.shape

(dtype('float32'), (8, 10, 64, 64, 3))

In [32]:
model = SIMONeModel(
    beta_o=1e-8,
    beta_f=1e-8,
#     optimizer=optax.adamw(lr),
#     eager=eager,
)
model = eg.load(path="runs/2021-11-25 17:17:46/")

In [33]:
rec_x, masks = model.predict(batch)
rec_x.shape

tcmalloc: large alloc 5431631872 bytes == 0x56e1a8000 @  0x7fd6e48ea680 0x7fd6e490b824 0x7fd6e490bb8a 0x7fd6d87a5157 0x7fd6d34d1df0 0x7fd6d34e0d94 0x7fd6d34e3c07 0x7fd6d3430eef 0x7fd6d3191d53 0x7fd6d317a846 0x5f5db9 0x5f698e 0x50b4c7 0x570e46 0x56a0ba 0x5f70bb 0x66600d 0x5f574e 0x56d5f6 0x56a0ba 0x5f6343 0x5f56c7 0x56d5f6 0x5f6166 0x56bf09 0x5f6166 0x56bf09 0x56a0ba 0x5f6343 0x5f56c7 0x56d5f6


(8, 16, 10, 64, 64, 3)

In [34]:
rec = rec_x * masks
rec.shape

(8, 16, 10, 64, 64, 3)

In [35]:
rec_comb = jnp.sum(rec, axis=1)
rec_comb.shape

(8, 10, 64, 64, 3)

In [36]:
videos = {}
for b, (x, comb, slots) in enumerate(zip(batch, rec_comb, rec_x)):
    videos[f"orig{b}"] = x
    videos[f"rec{b}"] = comb
    for k, slot in enumerate(slots):
        videos[f"{b}slot{k+1}"] = slot
media.show_videos(videos, fps=10, columns=18)

In [22]:
object_posterior, frame_posterior = model.encode(batch)
object_means, object_logstd = object_posterior.loc, object_posterior.scale_diag
frame_means, frame_logstd = frame_posterior.loc, frame_posterior.scale_diag

In [23]:
import distrax as dist

In [24]:
# Same objects, different frames
b, k, c = object_means.shape
b, t, c = frame_means.shape

o1_posterior = dist.MultivariateNormalDiag(
    E.repeat(object_means[0], 'k c -> b k c', b=b),
    E.repeat(object_logstd[0], 't c -> b t c', b=b)
)


In [25]:
rec_o1, masks_o1 = model.decode(o1_posterior, frame_posterior, batch)
comb_o1 = jnp.sum(rec_o1 * masks_o1, axis=1)

In [37]:
videos = {}
for i, x in enumerate(batch):
    videos[f"orig{i}"] = x
for i, comb in enumerate(comb_o1):
    videos[f"rec{i}"] = comb
media.show_videos(videos, height=128, fps=10, columns=len(batch))

"orig2 <source src=""data:video/mp4;base64,AAAAIGZ0eXBpc29tAAACAGlzb21pc28yYXZjMW1wNDEAAAAIZnJlZQAABextZGF0AAACfwYF//973EXpvebZSLeWLNgg2SPu73gyNjQgLSBjb3JlIDE1NSByMjkxNyAwYTg0ZDk4IC0gSC4yNjQvTVBFRy00IEFWQyBjb2RlYyAtIENvcHlsZWZ0IDIwMDMtMjAxOCAtIGh0dHA6Ly93d3cudmlkZW9sYW4ub3JnL3gyNjQuaHRtbCAtIG9wdGlvbnM6IGNhYmFjPTEgcmVmPTMgZGVibG9jaz0xOjA6MCBhbmFseXNlPTB4MzoweDExMyBtZT1oZXggc3VibWU9NyBwc3k9MSBwc3lfcmQ9MS4wMDowLjAwIG1peGVkX3JlZj0xIG1lX3JhbmdlPTE2IGNocm9tYV9tZT0xIHRyZWxsaXM9MSA4eDhkY3Q9MSBjcW09MCBkZWFkem9uZT0yMSwxMSBmYXN0X3Bza2lwPTEgY2hyb21hX3FwX29mZnNldD0tMiB0aHJlYWRzPTIgbG9va2FoZWFkX3RocmVhZHM9MSBzbGljZWRfdGhyZWFkcz0wIG5yPTAgZGVjaW1hdGU9MSBpbnRlcmxhY2VkPTAgYmx1cmF5X2NvbXBhdD0wIGNvbnN0cmFpbmVkX2ludHJhPTAgYmZyYW1lcz0zIGJfcHlyYW1pZD0yIGJfYWRhcHQ9MSBiX2JpYXM9MCBkaXJlY3Q9MSB3ZWlnaHRiPTEgb3Blbl9nb3A9MCB3ZWlnaHRwPTIga2V5aW50PTI1MCBrZXlpbnRfbWluPTEwIHNjZW5lY3V0PTQwIGludHJhX3JlZnJlc2g9MCByYz1jcXAgbWJ0cmVlPTAgcXA9MjAgaXBfcmF0aW89MS40MCBwYl9yYXRpbz0xLjMwIGFxPTAAgAAAAQBliIQA//69Z+ZWxLUrH26xx00Fs2QyXmZWvg7orvROCah+46ZyepDrVYDadscOIbVgnN13wbr3J6OO67jdeOjyoejk43EqSN3+DApK2LlKPpBU1vI7rgauTB5FTK+kpbCMFiriU7rh9SyVAxpAhnn6qhfIdldYZUDM3VWlnLn6kjcVZEySYg8MCQk2lsYKtC83JWiz37NtLxVJ1I6tEcZxI2T/P+azE/B1nhx1GV7f6+Oa+4MDQQXNIKLT6eM8fMxeGEhRZZCJEg3KVYQgVHX/to+dkpY+2qw3Tr7CC2xYXdebyOaSZLaEu9qmFWFoxS5GxavE3tfGqv7qGs5kDV2xAAAAZUGaIWx/5U9LPSHNhY87aaNC9yKsUulYweTdAyLQHvafbsWk2G6FPO8VQXrosD6HXA6KgpxmOtgXxrtv8O1L7Ha1Qk9E0ZfdIy6TyKv+uPv0J/Fbefc/6L2vLc616fRhzqWaf7vAAAAAWEGaQjwhkymH/+T5s/JNOHo4cbLq9gQQ2eVYWk3oeegsaqffgpK6kU1W597tplne+FFtEPyZ/RoRCmxvzlitwGfNwQP7dR94jZFqteyjs1lFcj+YIkG7bvEAAABRQZpkSeEPJlMFP//k/PXvTh9Ou3pBKPpwXE0NZkdOVrlmelp3vn6zj5TENhzW/Puq8AEzQMKCZUCYX4GKU9Pml0Sqga/ICv9JYgYIlQZ+2ju8AAAAQAGeg2pJ/1V8CQgRLnV0E/5miZD2msdnxROPu/DAg1UeaL9Br3V4vzdbsyCBMwSZEPOQmm+9GVBnCZBo2jF3YucAAAApQZqISeEPJlMD/+T7XbQoTLtxzecDfI94XE5Io/9uGjcsp1/GVmH8u3kAAAAsQZ6mRRE9f0xuVpKOKxzgvWVJjcuP0Etd25XLTOYqQ5fTrF0x7LWfMwwI/LEAAAAmAZ7FdEn/VoMLbB6mm6QIWgqaG297+MibejvYVd+hWAnMubD85YEAAAAzAZ7Hakn/VX9dgxip3AcF5J/xL6i+XMVjv814CdyNXeqtDi1UVI53apDKC0XUs9nCHzXAAAAAPUGayUmoQWiZTA//5VGh6S+V5f7HLvaal2H9zoIVcxROUSeTierq4OJy5lbFNkE8QcL5FjHJqy4EzYjP3eAAAAN5bW9vdgAAAGxtdmhkAAAAAAAAAAAAAAAAAAAD6AAAA+gAAQAAAQAAAAAAAAAAAAAAAAEAAAAAAAAAAAAAAAAAAAABAAAAAAAAAAAAAAAAAABAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAgAAAqN0cmFrAAAAXHRraGQAAAADAAAAAAAAAAAAAAABAAAAAAAAA+gAAAAAAAAAAAAAAAAAAAAAAAEAAAAAAAAAAAAAAAAAAAABAAAAAAAAAAAAAAAAAABAAAAAAEAAAABAAAAAAAAkZWR0cwAAABxlbHN0AAAAAAAAAAEAAAPoAAAIAAABAAAAAAIbbWRpYQAAACBtZGhkAAAAAAAAAAAAAAAAAAAoAAAAKABVxAAAAAAALWhkbHIAAAAAAAAAAHZpZGUAAAAAAAAAAAAAAABWaWRlb0hhbmRsZXIAAAABxm1pbmYAAAAUdm1oZAAAAAEAAAAAAAAAAAAAACRkaW5mAAAAHGRyZWYAAAAAAAAAAQAAAAx1cmwgAAAAAQAAAYZzdGJsAAAAlnN0c2QAAAAAAAAAAQAAAIZhdmMxAAAAAAAAAAEAAAAAAAAAAAAAAAAAAAAAAEAAQABIAAAASAAAAAAAAAABAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAGP//AAAAMGF2Y0MBZAAK/+EAF2dkAAqs2UQmhAAAAwAEAAADAFA8SJZYAQAGaOvhssiwAAAAGHN0dHMAAAAAAAAAAQAAAAoAAAQAAAAAFHN0c3MAAAAAAAAAAQAAAAEAAABQY3R0cwAAAAAAAAAIAAAAAwAACAAAAAABAAAMAAAAAAEAAAQAAAAAAQAAFAAAAAABAAAIAAAAAAEAAAAAAAAAAQAABAAAAAABAAAIAAAAABxzdHNjAAAAAAAAAAEAAAABAAAACgAAAAEAAAA8c3RzegAAAAAAAAAAAAAACgAAA4cAAABpAAAAXAAAAFUAAABEAAAALQAAADAAAAAqAAAANwAAAEEAAAAUc3RjbwAAAAAAAAABAAAAMAAAAGJ1ZHRhAAAAWm1ldGEAAAAAAAAAIWhkbHIAAAAAAAAAAG1kaXJhcHBsAAAAAAAAAAAAAAAALWlsc3QAAAAlqXRvbwAAAB1kYXRhAAAAAQAAAABMYXZmNTguMjkuMTAw"" type=""video/mp4""/> This browser does not support the video tag.","orig4 <source src=""data:video/mp4;base64,AAAAIGZ0eXBpc29tAAACAGlzb21pc28yYXZjMW1wNDEAAAAIZnJlZQAABb5tZGF0AAACfwYF//973EXpvebZSLeWLNgg2SPu73gyNjQgLSBjb3JlIDE1NSByMjkxNyAwYTg0ZDk4IC0gSC4yNjQvTVBFRy00IEFWQyBjb2RlYyAtIENvcHlsZWZ0IDIwMDMtMjAxOCAtIGh0dHA6Ly93d3cudmlkZW9sYW4ub3JnL3gyNjQuaHRtbCAtIG9wdGlvbnM6IGNhYmFjPTEgcmVmPTMgZGVibG9jaz0xOjA6MCBhbmFseXNlPTB4MzoweDExMyBtZT1oZXggc3VibWU9NyBwc3k9MSBwc3lfcmQ9MS4wMDowLjAwIG1peGVkX3JlZj0xIG1lX3JhbmdlPTE2IGNocm9tYV9tZT0xIHRyZWxsaXM9MSA4eDhkY3Q9MSBjcW09MCBkZWFkem9uZT0yMSwxMSBmYXN0X3Bza2lwPTEgY2hyb21hX3FwX29mZnNldD0tMiB0aHJlYWRzPTIgbG9va2FoZWFkX3RocmVhZHM9MSBzbGljZWRfdGhyZWFkcz0wIG5y

In [28]:
# Different objects, same frames
b, k, c = object_means.shape
b, t, c = frame_means.shape

f1_posterior = dist.MultivariateNormalDiag(
    E.repeat(frame_means[0], 't c -> b t c', b=b),
    E.repeat(frame_logstd[0], 't c -> b t c', b=b)
)


In [30]:
rec_f1, masks_f1 = model.decode(object_posterior, f1_posterior, batch)
comb_f1 = jnp.sum(rec_f1 * masks_f1, axis=1)

In [31]:
videos = {}
for i, x in enumerate(batch):
    videos[f"orig{i}"] = x
for i, comb in enumerate(comb_f1):
    videos[f"rec{i}"] = comb
media.show_videos(videos, height=128, fps=10, columns=len(batch))